# Atividade Spark 

Considerando o dataset detalhado a seguir, extraia o conjunto de informações solicitadas.

### Dataset dados de COVID no Brasil

- Arquivo disponível em /home/dados/covid/covid.csv
- Dados relativos a pacientes que realizaram exames de Covid19 no Brasil
- ~1.6GB
- 4.4M de instâncias


|  # | Nome do campo        	| Descrição                                     	|
|----|----------------------	|-----------------------------------------------	|
|  0 | id                   	| identificador                                 	|
|  1 | dataNotificacao      	| Data da notificação                           	|
|  2 | dataInicioSintomas   	| Data do inicio dos sintomas                   	|
|  3 | dataNascimento       	| Data de nascimento                            	|
|  4 | sintomas             	| Sintomas do paciente                          	|
|  5 | profissionalSaude    	| Relacionado a profissional de saúde           	|
|  6 | cbo                  	| Ocupação                                      	|
|  7 | condicoes            	| Condições do paciente                         	|
|  8 | estadoTeste          	| Estado do teste                               	|
|  9 | dataTeste            	| Data do teste                                 	|
| 10 | tipoTeste            	| Tipo de teste realizado                       	|
| 11 | resultadoTeste       	| Resultado do Teste                            	|
| 12 | paisOrigem           	| Pais de Origem do paciente                    	|
| 13 | sexo                 	| Sexo do paciente                              	|
| 14 | bairro               	| Bairro do paciente                            	|
| 15 | estado               	| Estado do paciente                            	|
| 16 | estadoIBGE              	| Estado do paciente IBGE                          	|
| 17 | municipio            	| Municipio do paciente                         	|
| 18 | municipioIBGE          	| Municipio do paciente                         	|
| 19 | cep            	        | CEP                                            	|
| 20 | origem               	| Origem do paciente                            	|
| 21 | cnes                 	| Código da unidade de saúde                    	|
| 22 | estadoNotificacao    	| Estado da notificação                         	|
| 23 | estadoNotificacaoIBGE    | Estado da notificação IBGE                      	|
| 24 | municipioNotificacao 	| Município da notificação                      	|
| 25 | municipioNotificacaoIBGE | Município da notificação IBGE                    	|
| 26 | numeroNotificacao    	| Número da notificação                         	|
| 27 | excluido             	| ID de exclusão                                	|
| 28 | validado             	| Local validação                               	|
| 29 | idade                	| Idade do paciente                             	|
| 30 | dataEncerramento     	| Data do encerramento da avaliação do paciente 	|
| 31 | evolucaoCaso         	| Evolução do caso do paciente                  	|
| 32 | classificacaoFinal   	| Avaliação final do caso                       	|

Informações a serem extraídas:

1. Quantidade de pacientes positivos para corona virus (resultadoTeste)
2. Quantidade de pacientes do sexo feminino positivos para corona virus (resultadoTeste)
3. Quantidade de pacientes de acordo com o sexo e o resultado do teste (resultadoTeste)
4. Sintomas mais comuns para casos positivos para corona virus
5. Sintomas mais comuns para casos não positivos para corona virus
6. Quantidade de casos positivos no Paraná
7. Municipio do Paraná com a maior quantidade de óbitos (evolucaoCaso)
8. Quantidade de municipios no Paraná com casos positivos
9. Quantidade de municipios no Paraná sem casos positivos
10. Estado com a maior taxa de falecimento de mulheres
11. Menor idade de mulher positiva para covid
12. Maior idade de mulher positiva para covid
13. Quantidade de casos positivos, em granularidade diária, ao longo do tempo 
14. Quantidade de casos positivos, em granularidade semanal, ao longo do tempo 
15. Quantidade de pacientes que faleceram (evolucaoCaso) separados pela idade
16. Taxa de óbito (evolucaoCaso) por idade
17. Idade média das mulheres positivas para covid
18. Município do Paraná com a maior quantidade de mulheres positivos para covid
19. Dia da semana com a maior quantidade de testes realizados
20. Dia da semana com maior quantidade de pacientes positivos para corona virus
21. Municipio, com mais de 500 testes, com a maior taxa de exames que não foram positivos para COVID (casos que foram testados porém não tiveram o resultadoTeste como Positivo)

**Dicas:**
- *Crie uma célula (Insert -> Insert Cell Below) para cada informação solicitada*
- *A análise deve ser feita sobre os dados do HDFS*
- *Inicialize o seu cluster executando o script em: Desktop/ambientes/spark/inicializar.sh*
- *Acesse o seu cluster executando o script em: Desktop/ambientes/spark/abrir_navegador.sh*

In [1]:
# abre sessão no spark
import os
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

import pyspark
conf = pyspark.SparkConf()

conf.setMaster('spark://spark-master:7077')

sc = pyspark.SparkContext.getOrCreate()
sc.stop()
sc = pyspark.SparkContext(conf = conf)

In [2]:
#carrega arquivo do HDFS em um RDD
arquivoRDD = sc.textFile('hdfs://namenode:9000/covid.csv')

In [3]:
#informacao 1 
# Quantidade de pacientes positivos para corona virus (resultadoTeste)
arquivoRDD.map(lambda l: [l.split(";")[11], 1])\
        .reduceByKey(lambda x, y: x+y)\
        .collect()

[('null', 646421),
 ('Negativo', 2547942),
 ('Positivo', 1230210),
 ('""', 31),
 ('resultadoTeste', 1)]

In [4]:
#informacao 2
# Quantidade de pacientes do sexo feminino positivos para corona virus (resultadoTeste)
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[13], 1])\
        .filter(lambda f: f[1]=="Feminino" and f[0]=="Positivo")\
        .map(lambda x: [x[0], x[2]])\
        .reduceByKey(lambda x,y: x+y)\
        .collect()

[('Positivo', 632899)]

In [5]:
#informacao 3 
# Quantidade de pacientes de acordo com o sexo e o resultado do teste (resultadoTeste)
arquivoRDD.map(lambda l: [l.split(";")[11]+" -- "+l.split(";")[13], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .collect()

[('"" -- Masculino', 4),
 ('resultadoTeste -- sexo', 1),
 ('Negativo -- Masculino', 1141559),
 ('Positivo -- Masculino', 578177),
 ('Negativo -- Indefinido', 88174),
 ('null -- Masculino', 274303),
 ('Positivo -- null', 17),
 ('Negativo -- Feminino', 1318173),
 ('Positivo -- Feminino', 632899),
 ('"" -- Feminino', 27),
 ('Negativo -- null', 36),
 ('null -- Indefinido', 4),
 ('Positivo -- Indefinido', 19117),
 ('null -- Feminino', 371661),
 ('null -- null', 453)]

In [6]:
#informacao 4
# Sintomas mais comuns para casos positivos para corona virus
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[4]])\
        .filter(lambda x: x[0]=="Positivo")\
        .map(lambda x: [x[1], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda x: x[1], False)\
        .collect()

[('Outros', 385582),
 ('Tosse, Outros', 73863),
 ('Tosse', 62818),
 ('Febre, Tosse, Outros', 61908),
 ('Febre, Outros', 54497),
 ('Febre, Tosse', 43796),
 ('Febre', 28205),
 ('Febre, Tosse, Dor de Garganta', 26261),
 ('Outros: Paciente assintomático', 24481),
 ('Tosse, Dor de Garganta', 18628),
 ('Dor de Garganta', 16393),
 ('Dor de Garganta, Outros', 16323),
 ('""', 14236),
 ('Febre, Tosse, Dispneia', 14222),
 ('Febre, Tosse, Outros, Dor de Garganta', 13506),
 ('Tosse, Febre, Outros', 12817),
 ('Tosse, Febre', 12333),
 ('Tosse, Outros, Dor de Garganta', 12221),
 ('Tosse, Dor de Garganta, Outros', 11433),
 ('Febre, Tosse, Dor de Garganta, Outros', 11394),
 ('Outros, Tosse', 10977),
 ('Febre, Tosse, Dor de Garganta, Dispneia', 10863),
 ('Outros, Dor de Garganta', 10781),
 ('Tosse, Dispneia', 10639),
 ('null', 10439),
 ('Dispneia', 9281),
 ('Dor de Garganta, Tosse, Outros', 9145),
 ('Febre, Dor de Garganta', 8414),
 ('Outros, Febre', 8146),
 ('Febre, Tosse, Outros, Dispneia', 8016),
 ('D

In [7]:
#informacao 5
# Sintomas mais comuns para casos não positivos para corona virus
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[4]])\
        .filter(lambda x: x[0]=="Negativo")\
        .map(lambda x: [x[1], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda x: x[1], False)\
        .collect()

[('Outros', 1101930),
 ('Outros: Paciente assintomático', 260856),
 ('Tosse', 120996),
 ('Tosse, Outros', 92789),
 ('Dor de Garganta', 59950),
 ('Febre, Outros', 51551),
 ('Febre, Tosse, Outros', 45485),
 ('""', 42022),
 ('Febre, Tosse', 41949),
 ('Dor de Garganta, Outros', 38446),
 ('Tosse, Dor de Garganta', 37532),
 ('Febre', 35993),
 ('Febre, Tosse, Dor de Garganta', 30474),
 ('Tosse, Dor de Garganta, Outros', 22867),
 ('Outros, Dor de Garganta', 22645),
 ('Tosse, Outros, Dor de Garganta', 20769),
 ('Dor de Garganta, Tosse, Outros', 20353),
 ('Dispneia', 20155),
 ('Tosse, Dispneia', 20016),
 ('Dor de Garganta, Tosse', 18242),
 ('Febre, Dor de Garganta', 15216),
 ('Dispneia, Outros', 14453),
 ('Febre, Tosse, Outros, Dor de Garganta', 13273),
 ('Febre, Tosse, Dor de Garganta, Outros', 13049),
 ('Tosse, Febre', 12937),
 ('Outros, Tosse', 12713),
 ('Febre, Tosse, Dor de Garganta, Dispneia', 12526),
 ('Febre, Tosse, Dispneia', 12360),
 ('Tosse, Dispneia, Outros', 11965),
 ('Febre, Dor de

In [8]:
#informacao 6
# Quantidade de casos positivos no Paraná
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[15]])\
        .filter(lambda x: x[0]=="Positivo" and x[1]=="PARANÁ")\
        .count()

6183

In [9]:
#informacao 7
# Municipio do Paraná com a maior quantidade de óbitos (evolucaoCaso)
arquivoRDD.map(lambda l: [l.split(";")[15], l.split(";")[17], l.split(";")[31]])\
        .filter(lambda f: f[0]=="PARANÁ" and f[2]=="Óbito")\
        .map(lambda c: [c[0]+" -- "+c[1], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda x: x[1], False)\
        .collect()

[('PARANÁ -- Foz do Iguaçu', 25),
 ('PARANÁ -- Curitiba', 14),
 ('PARANÁ -- Pinhais', 5),
 ('PARANÁ -- Toledo', 4),
 ('PARANÁ -- Londrina', 4),
 ('PARANÁ -- Mariluz', 4),
 ('PARANÁ -- Mandirituba', 3),
 ('PARANÁ -- Paranavaí', 3),
 ('PARANÁ -- Irati', 3),
 ('PARANÁ -- Cambé', 3),
 ('PARANÁ -- Diamante do Sul', 2),
 ('PARANÁ -- Umuarama', 2),
 ('PARANÁ -- Guaíra', 2),
 ('PARANÁ -- Corbélia', 2),
 ('PARANÁ -- Jaguapitã', 2),
 ('PARANÁ -- Santa Terezinha de Itaipu', 2),
 ('PARANÁ -- Ivaiporã', 2),
 ('PARANÁ -- Piraquara', 2),
 ('PARANÁ -- Quitandinha', 2),
 ('PARANÁ -- São João do Ivaí', 2),
 ('PARANÁ -- Astorga', 1),
 ('PARANÁ -- Campo Do Tenente', 1),
 ('PARANÁ -- Pérola', 1),
 ('PARANÁ -- Cafelândia', 1),
 ('PARANÁ -- Santo Antônio da Platina', 1),
 ('PARANÁ -- Matinhos', 1),
 ('PARANÁ -- Rio Branco Do Sul', 1),
 ('PARANÁ -- Adrianópolis', 1),
 ('PARANÁ -- Campo Bonito', 1),
 ('PARANÁ -- Japurá', 1),
 ('PARANÁ -- Cascavel', 1),
 ('PARANÁ -- Campo do Tenente', 1),
 ('PARANÁ -- São Migue

In [10]:
#informacao 8
# Quantidade de municipios no Paraná com casos positivos
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[15], l.split(";")[17]])\
        .filter(lambda f: f[0]=="Positivo" and f[1]=="PARANÁ")\
        .map(lambda x: [x[2],1])\
        .reduceByKey(lambda x,y: x+y)\
        .count()

296

In [11]:
#informacao 9
# Quantidade de municipios no Paraná sem casos positivos
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[15], l.split(";")[17]])\
        .filter(lambda f: f[0]=="Negativo" and f[1]=="PARANÁ")\
        .map(lambda x: [x[2],1])\
        .reduceByKey(lambda x,y: x+y)\
        .count()

407

In [12]:
#informacao 10
# Estado com a maior taxa de falecimento de mulheres
arquivoRDD.map(lambda l: [l.split(";")[13], l.split(";")[15], l.split(";")[31]])\
        .filter(lambda f: f[0]=="Feminino" and f[2]=="Óbito")\
        .map(lambda c: [c[1], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda s: s[1], False)\
        .collect()

[('BAHIA', 1588),
 ('SÃO PAULO', 878),
 ('RIO DE JANEIRO', 743),
 ('MINAS GERAIS', 430),
 ('SANTA CATARINA', 181),
 ('RIO GRANDE DO SUL', 115),
 ('PARANÁ', 59)]

In [13]:
#informacao 11
# Menor idade de mulher positiva para covid

arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[13], l.split(";")[29]])\
        .filter(lambda f: f[0]=="Positivo" and f[1]=="Feminino")\
        .sortBy(lambda s: s[2], True)\
        .first()

['Positivo', 'Feminino', '0']

In [14]:
#informacao 12
# Maior idade de mulher positiva para covid
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[13], l.split(";")[29]])\
        .filter(lambda f: f[0]=="Positivo" and f[1]=="Feminino")\
        .sortBy(lambda s: s[2], False)\
        .first()

['Positivo', 'Feminino', '99']

In [15]:
#informacao 13
# Quantidade de casos positivos, em granularidade diária, ao longo do tempo 
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[9].split("T")[0]])\
        .filter(lambda f: f[0]=="Positivo")\
        .map(lambda c: [c[1], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda s: s[0], True)\
        .collect()

[('2020-01-01', 13),
 ('2020-01-02', 3),
 ('2020-01-06', 3),
 ('2020-01-07', 2),
 ('2020-01-08', 1),
 ('2020-01-09', 1),
 ('2020-01-10', 1),
 ('2020-01-13', 4),
 ('2020-01-15', 5),
 ('2020-01-16', 1),
 ('2020-01-17', 5),
 ('2020-01-18', 1),
 ('2020-01-20', 8),
 ('2020-01-21', 5),
 ('2020-01-22', 4),
 ('2020-01-23', 2),
 ('2020-01-24', 2),
 ('2020-01-25', 1),
 ('2020-01-27', 4),
 ('2020-01-28', 5),
 ('2020-01-29', 4),
 ('2020-01-30', 8),
 ('2020-01-31', 8),
 ('2020-02-01', 34),
 ('2020-02-02', 24),
 ('2020-02-03', 16),
 ('2020-02-04', 13),
 ('2020-02-05', 10),
 ('2020-02-06', 22),
 ('2020-02-07', 11),
 ('2020-02-08', 4),
 ('2020-02-09', 6),
 ('2020-02-10', 11),
 ('2020-02-11', 12),
 ('2020-02-12', 9),
 ('2020-02-13', 17),
 ('2020-02-14', 19),
 ('2020-02-15', 32),
 ('2020-02-16', 25),
 ('2020-02-17', 26),
 ('2020-02-18', 39),
 ('2020-02-19', 29),
 ('2020-02-20', 30),
 ('2020-02-21', 13),
 ('2020-02-22', 23),
 ('2020-02-23', 12),
 ('2020-02-24', 32),
 ('2020-02-25', 30),
 ('2020-02-26', 2

In [16]:
"""

CÓDIGO UTILIZADO NA INFORMAÇÃO 14.

"""
daysIn = lambda x: 31 if x % 2 != 0 else (28 if x==2 else 30)

_a,_m, datesPossible, i, lim = 2020, 1, [], 0, daysIn(1)
while True:
    i += 1
    if _m == 9:
        break
    
    if i <= lim:
        datesPossible.append(str(_a)+"-0"+str(_m)+"-"+("" if i > 9 else "0")+str(i))
    
    if i >= daysIn(_m):
        _m += 1
        i = 0
    
#print(datesPossible)


# Defining week number:
# Primeiro domingo do ano foi 5 de janeiro.

weekAt = dict()

k, tr, wUpdate = 0, 0, 2
while k < len(datesPossible):
    if k < 4:
        weekAt[datesPossible[k]] = 1
        k += 1
        continue
    if tr == 7:
        wUpdate += 1
        tr = 0
    weekAt[datesPossible[k]] = wUpdate
    k += 1
    tr += 1
    
#print(weekAt)
print(weekAt['2020-01-11'])

2


In [17]:
#informacao 14
# Quantidade de casos positivos, em granularidade semanal, ao longo do tempo 
# Primeiro domingo do ano foi dia 5 de janeiro => Week 2 -> de 5 a 11 de janeiro.
# Use   print(weekAt['2020-01-11'])  para checar a semana correspondente.

arquivoRDD.map(lambda l: [l.split(";")[11], (l.split(";")[9].split("T")[0])])\
        .filter(lambda f: f[0]=="Positivo" and f[1] != "data")\
        .filter(lambda f: f[1] in weekAt.keys())\
        .map(lambda c: [weekAt[c[1]], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda s: s[0], True)\
        .collect()               

[(1, 16),
 (2, 8),
 (3, 16),
 (4, 22),
 (5, 63),
 (6, 100),
 (7, 106),
 (8, 185),
 (9, 148),
 (10, 157),
 (11, 668),
 (12, 2173),
 (13, 4915),
 (14, 8286),
 (15, 10523),
 (16, 16005),
 (17, 19305),
 (18, 36158),
 (19, 35005),
 (20, 43421),
 (21, 58017),
 (22, 66418),
 (23, 71537),
 (24, 77817),
 (25, 99288),
 (26, 114098),
 (27, 114711),
 (28, 130998),
 (29, 135211),
 (30, 118730),
 (31, 66095),
 (32, 1)]

In [18]:
#informacao 15
# Quantidade de pacientes que faleceram (evolucaoCaso) separados pela idade
arquivoRDD.map(lambda l: [l.split(";")[31], l.split(";")[29]])\
        .filter(lambda f: f[0]=="Óbito")\
        .map(lambda c: [int(c[1]), 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda s: s[0])\
        .collect()

[(0, 271),
 (1, 9),
 (2, 14),
 (3, 7),
 (4, 5),
 (5, 3),
 (6, 4),
 (7, 6),
 (8, 2),
 (9, 7),
 (10, 3),
 (12, 4),
 (13, 2),
 (14, 2),
 (15, 3),
 (16, 12),
 (17, 7),
 (18, 10),
 (19, 14),
 (20, 14),
 (21, 16),
 (22, 21),
 (23, 15),
 (24, 15),
 (25, 23),
 (26, 33),
 (27, 24),
 (28, 28),
 (29, 25),
 (30, 27),
 (31, 19),
 (32, 39),
 (33, 43),
 (34, 51),
 (35, 43),
 (36, 55),
 (37, 65),
 (38, 56),
 (39, 66),
 (40, 54),
 (41, 61),
 (42, 74),
 (43, 82),
 (44, 75),
 (45, 78),
 (46, 91),
 (47, 83),
 (48, 85),
 (49, 77),
 (50, 83),
 (51, 88),
 (52, 107),
 (53, 105),
 (54, 126),
 (55, 140),
 (56, 158),
 (57, 124),
 (58, 134),
 (59, 161),
 (60, 153),
 (61, 178),
 (62, 169),
 (63, 189),
 (64, 173),
 (65, 170),
 (66, 193),
 (67, 209),
 (68, 205),
 (69, 201),
 (70, 210),
 (71, 230),
 (72, 195),
 (73, 216),
 (74, 195),
 (75, 200),
 (76, 212),
 (77, 181),
 (78, 174),
 (79, 199),
 (80, 212),
 (81, 183),
 (82, 185),
 (83, 175),
 (84, 169),
 (85, 159),
 (86, 139),
 (87, 151),
 (88, 133),
 (89, 112),
 (90, 

In [19]:
#informacao 16
# Taxa de óbito (evolucaoCaso) por idade

def order16(data):
    up = 1 if data[0] == "Óbito" else 0
    return [1, up, data[1]] # [total, is_dead, age]

def reduce16(x, y):
    return [x[0]+y[0], x[1]+y[1]]

def taxa16(x):
    return "{:.2f}".format(x[1]*100.0/x[0]*1.0)+"%"

arquivoRDD.map(lambda l: [l.split(";")[31], l.split(";")[29]])\
        .filter(lambda f: len(f[0]) > 4 and f[1]!='idade' and int(f[1]) < 111)\
        .map(lambda c: order16(c))\
        .map(lambda c: [c[2],[c[0], c[1]]])\
        .reduceByKey(lambda x,y: reduce16(x,y))\
        .map(lambda x: [int(x[0]), taxa16(x[1])])\
        .sortBy(lambda s: s[0], True)\
        .collect()
    


[[0, '9.88%'],
 [1, '0.53%'],
 [2, '0.95%'],
 [3, '0.59%'],
 [4, '0.42%'],
 [5, '0.27%'],
 [6, '0.33%'],
 [7, '0.50%'],
 [8, '0.17%'],
 [9, '0.57%'],
 [10, '0.25%'],
 [11, '0.00%'],
 [12, '0.31%'],
 [13, '0.14%'],
 [14, '0.13%'],
 [15, '0.16%'],
 [16, '0.58%'],
 [17, '0.29%'],
 [18, '0.29%'],
 [19, '0.28%'],
 [20, '0.22%'],
 [21, '0.23%'],
 [22, '0.27%'],
 [23, '0.19%'],
 [24, '0.18%'],
 [25, '0.24%'],
 [26, '0.34%'],
 [27, '0.24%'],
 [28, '0.28%'],
 [29, '0.23%'],
 [30, '0.25%'],
 [31, '0.16%'],
 [32, '0.32%'],
 [33, '0.34%'],
 [34, '0.40%'],
 [35, '0.33%'],
 [36, '0.42%'],
 [37, '0.47%'],
 [38, '0.39%'],
 [39, '0.48%'],
 [40, '0.41%'],
 [41, '0.49%'],
 [42, '0.61%'],
 [43, '0.73%'],
 [44, '0.71%'],
 [45, '0.76%'],
 [46, '0.92%'],
 [47, '0.89%'],
 [48, '0.94%'],
 [49, '0.87%'],
 [50, '1.01%'],
 [51, '1.10%'],
 [52, '1.41%'],
 [53, '1.38%'],
 [54, '1.78%'],
 [55, '2.00%'],
 [56, '2.32%'],
 [57, '2.04%'],
 [58, '2.33%'],
 [59, '2.99%'],
 [60, '3.09%'],
 [61, '3.83%'],
 [62, '3.89%'],
 [

In [20]:
#informacao 17
# Idade média das mulheres positivas para covid
checkpoint17 = arquivoRDD.map(lambda l: [l.split(";")[13], l.split(";")[11], l.split(";")[29]])\
                    .filter(lambda f: f[0]=="Feminino" and f[1]=="Positivo")

qntd = checkpoint17.count()
soma = checkpoint17.map(lambda f: ['soma', int(f[2])]).reduceByKey(lambda x,y: x+y).collect()

print("Idade média:  "+str(int(soma[0][1])*1.0/qntd*1.0))

Idade média:  40.96124658120806


In [21]:
#informacao 18
# Município do Paraná com a maior quantidade de mulheres positivos para covid
arquivoRDD.map(lambda l: [l.split(";")[11], l.split(";")[13], l.split(";")[15], l.split(";")[17]])\
        .filter(lambda f: f[2]=="PARANÁ" and f[1]=="Feminino" and f[0]=="Positivo")\
        .map(lambda c: [c[3], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda s: s[1], False)\
        .first()

('Curitiba', 903)

In [22]:
"""

CÓDIGO UTILIZADO DA INFORMAÇÃO 19.

"""

# Dia da Semana => Dom = 0, Seg = 1, ... Sáb = 6.

# Primeiro dia do ano foi em uma quarta-feira.
# Usando o dicionário da questão 14 como auxiliar.
dia = 3

dayOfTheWeekAt = dict()

for x in weekAt.keys():
    if dia == 7:
        dia = 0
    dayOfTheWeekAt[x] = dia
    dia += 1

ans = [(0, "Domingo"), (1, "Segunda"), (2,"Terça"), (3, "Quarta"), 
       (4,"Quinta"), (5, "Sexta"), (6, "Sábado")]

decodeDay = lambda x: ans[x][1]


In [23]:
#informacao 19
#Dia da semana com a maior quantidade de testes realizados

# Dia da Semana => Dom = 0, Seg = 1, ... Sáb = 6.

dday = arquivoRDD.map(lambda l: str(l.split(";")[9]).split("T")[0])\
        .filter(lambda f: f in weekAt.keys())\
        .map(lambda c: [dayOfTheWeekAt[c], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda x: x[1], False)\
        .first()[0]

print("Dia da semana com mais testes realizados: "+decodeDay(dday))

Dia da semana com mais testes realizados: Domingo


In [24]:
#informacao 20
# Dia da semana com maior quantidade de pacientes positivos para corona virus
dsem = arquivoRDD.map(lambda l: [str(l.split(";")[9]).split("T")[0], l.split(";")[11]])\
        .filter(lambda f: f[0] in weekAt.keys() and f[1]=="Positivo")\
        .map(lambda c: [dayOfTheWeekAt[c[0]], 1])\
        .reduceByKey(lambda x,y: x+y)\
        .sortBy(lambda x: x[1], False)\
        .first()[0]

print("Dia da semana com mais pacientes positivos para corona: "+decodeDay(dsem))

Dia da semana com mais pacientes positivos para corona: Domingo


In [34]:
#informacao 21
# Municipio, com mais de 500 testes, com a maior taxa de exames que não
# foram positivos para COVID (casos que foram testados porém não tiveram
# o resultadoTeste como Positivo)


# returns  [city, pos, total]
def order21(data):
    up = (1 if data[1] == "Positivo" else 0)
    return [data[0], up, 1]

def reduce21(x, y):
    return [x[0]+y[0], x[1]+y[1]]

# (total - positivies) / total
def taxa21(x):
    return "{:.2f}".format( ((x[1] - x[0])*100.0/x[1]*1.0) )+"%"

base = arquivoRDD.map(lambda l: [l.split(";")[17], l.split(";")[11]])

municipios500 = base.map(lambda c: [c[0], 1]).reduceByKey(lambda x,y: x+y)\
                .filter(lambda f: f[1]>500).map(lambda x: x[0]).collect()


winner = base.filter(lambda f: f[0] in municipios500)\
            .map(lambda c: order21(c))\
            .map(lambda c: [c[0], [c[1],c[2]]])\
            .reduceByKey(lambda x,y: reduce21(x, y))\
            .map(lambda c: [c[0], taxa21(c[1])])\
            .sortBy(lambda s: s[1], False)\
            .first()

winner

['Santo Antônio do Grama', '97.34%']